In [ ]:
import os
import numpy as np
import pandas as pd
from skimage import io, measure
import tifffile

import seaborn as sns

In [ ]:
ddir1 = "/u/home/f/f7xiesnm/project-zipursky/easifish/results/viz_all_projections_jan29"

dataset_name = 'cdf03_c1-2_bino_r1_autos1_flatfused_c0_s4'

# image 
f_img = os.path.join(ddir1, 'cdf03_c1-2_bino_r1_autos1_flatfused_c0_s4.tiff')
f_msk = '/u/home/f/f7xiesnm/project-zipursky/easifish/cdf03_c1-2_bino/outputs/r1/segmentation/r1-c3.tif'


# view 1 image
img = io.imread(f_img)
img = np.array(img)

msk = io.imread(f_msk)
msk = np.array(msk)

img.shape, msk.shape



In [ ]:
msk = msk[::2, ::2, ::2]
msk = msk[:,:-1,:-1]
msk.shape

In [ ]:
%%time
props = measure.regionprops_table(msk.astype(int), 
                                  intensity_image=img,
                                  properties=['centroid', 'area', 'mean_intensity']) # , 'max_intensity', 'min_intensity'
props = pd.DataFrame(props)
props

In [ ]:
intn = props['mean_intensity'].values
np.max(intn)

In [ ]:
sns.scatterplot(data=props, x='area', y='mean_intensity', s=1)

In [ ]:
sns.histplot(intn, bins=np.arange(0,80,2))

In [ ]:
sns.histplot(np.log2(1+intn)) # , bins=np.arange(0,80,2))

# visualize the masks
# use S3 later

In [ ]:
def masks_to_labeled_masks(msk, labeled_cells):
    unq, inv = np.unique(msk.reshape(-1,), return_inverse=True)

    for i in unq:
        if i not in labeled_cells:
            unq[i] = 0

    labeled_msk = unq[inv].reshape(msk.shape)
    
    return labeled_msk

In [ ]:
# keep the masks with high intensity
labeled_cells = 1+props[props['mean_intensity']>5].index.values
labeled_masks = masks_to_labeled_masks(msk, labeled_cells)
print(np.unique(labeled_msk).shape)

# save as tiff

# download and viz

In [ ]:
output = os.path.join(ddir1, f'{dataset_name}_labeled_masks.tiff')
print(output)
tifffile.imwrite(output, labeled_msk)